# Definición de parámetros

In [0]:
# catalog = "sales"
# bronze_schema = "sales_bronze"
# bronze_table = "sales_raw"

In [0]:
#Obtención de los parametros
try:
    catalog = dbutils.widgets.get("catalog")
    bronze_schema = dbutils.widgets.get("bronze_schema")
    bronze_table = dbutils.widgets.get("bronze_table")
    path_process = "/Volumes/sales/sales_bronze/files/process/"
    path_processed = "/Volumes/sales/sales_bronze/files/processed/"
    print("Parámetros cargados exitosamente.")

except Exception as e:
    print(f"Error fatal: No se pudieron obtener los parámetros")
    print(f"Detalle del error: {e}")
    # Detiene la ejecución del notebook si los parámetros fallan
    raise Exception("Error al obtener parámetros")


# Cargar Dataframe de Sales

In [0]:
import re
import os

try:
  files = dbutils.fs.ls(path_process)

  #Solo cargo archivos csv
  csv_files = [f.path for f in files if f.path.endswith('.csv')]

  #Si no hay archivos csv, termino el notebook
  if not csv_files:
    raise Exception("Files Not Found: No se encontraron archivos .csv en el directorio")

  #Analizo solamente los archivos que tengan estructura YYYYMMDD-YYYYMMDD_Sales.csv en su nombre
  pattern = re.compile(r".*/(\d{8})-(\d{8})_Sales\.csv$")

  matched_files = []
  for f in csv_files:
    if pattern.match(f):
      matched_files.append(f)
    else:
      print(f"El archivo {f} no coincide con el patrón de nombre y será ignorado.")

  #Si ningun archivo cumple con el patron, termino el notebook
  if not matched_files:
    raise Exception("Files Not Found: Ningún archivo CSV coincidió con el patrón de nombre requerido.")

  file_names = [os.path.basename(f) for f in matched_files]
  print(f"Archivos encontrados: {', '.join(file_names)}")
  df_raw = spark.read.csv(matched_files, header=True, inferSchema=False)
  print(f"Archivos cargados exitosamente: {', '.join(file_names)}")

except Exception as e:
    print(f"Error fatal: Fallo durante la búsqueda o lectura de archivos.")
    print(f"Detalle del error: {e}")
    # Detiene el notebook
    raise Exception("Error al leer los archivos de origen")


# Limpieza Data Frame 

In [0]:
try:
    from pyspark.sql.functions import expr

    #Dada la existencia de registros inválidos, se aplica un filtro para asegurar que solo se carguen registros válidos
    valid_cond = (
        # Columnas que deben ser de tipo entero
        expr("try_cast(category as int) is not null") &
        expr("try_cast(vendor_no as int) is not null") &
        expr("try_cast(itemno as int) is not null") &
        # Columns que deben ser de tipo string
        expr("try_cast(category_name as int) is null") &
        expr("try_cast(vendor_name as int) is null") &
        expr("try_cast(im_desc as int) is null") &
        expr("try_cast(county as int) is null")
    )

    df_final = df_raw.filter(valid_cond)
    print(f"Filtro completado. Registros válidos: {df_final.count()}")
except Exception as e:
    print(f"Error fatal: Fallo durante el filtrado de datos válidos.")
    print(f"Detalle del error: {e}")
    raise Exception("Error en la lógica de transformación. Verifique las condiciones del filtro.")



# Crear Raw_Sales si no existe

In [0]:
try:  
  #Determino las columnas y sus correspondientes tipos de datos dentro del df
  columns = [f"`{field.name}` {field.dataType.simpleString().upper()}" for field in df_final.schema.fields]
  columns.append("StoreDay TIMESTAMP")

  #Query de creacion de tabla en caso de que no exista
  create_query = f"""
  CREATE TABLE IF NOT EXISTS {catalog}.{bronze_schema}.{bronze_table} (
    {', '.join(columns)}
  )
  USING DELTA
  """
  spark.sql(create_query)
except Exception as e:
    print(f"Error fatal: Fallo durante la creación de la tabla.")
    print(f"Detalle del error: {e}")
    raise Exception("Verifique la sintaxis de CREATE TABLE")

# Insertar Datos 

In [0]:
try:  
  from pyspark.sql.functions import current_timestamp

  # Trunca la tabla destino antes de insertar los datos
  spark.sql(f"TRUNCATE TABLE {catalog}.{bronze_schema}.{bronze_table}")
  print("Tabla destino truncada.")

  # Agrega la columna StoreDay con el valor timestamp actual
  df_insert = df_final.withColumn("StoreDay", current_timestamp())

  # Define los nombres de columnas de destino en el orden correcto
  dest_cols = [
    "invoice_line_no",
    "date",
    "store",
    "name",
    "address",
    "city",
    "zipcode",
    "store_location",
    "county_number",
    "county",
    "category",
    "category_name",
    "vendor_no",
    "vendor_name",
    "itemno",
    "im_desc",
    "pack",
    "bottle_volume_ml",
    "state_bottle_cost",
    "state_bottle_retail",
    "sale_bottles",
    "sale_dollars",
    "sale_liters",
    "sale_gallons",
    "`:@computed_region_3r5t_5243`",
    "`:@computed_region_wnea_7qqw`",
    "`:@computed_region_i9mz_6gmt`",
    "`:@computed_region_uhgg_e8y2`",
    "`:@computed_region_e7ym_nrbf`",
    "StoreDay"
  ]

  # Crea una vista temporal
  df_insert.select(dest_cols).createOrReplaceTempView("tmp_sales_insert")

  # Inserta los datos usando SQL
  spark.sql(f"""
  INSERT INTO {catalog}.{bronze_schema}.{bronze_table} ({', '.join(dest_cols)})
  SELECT {', '.join(dest_cols)} FROM tmp_sales_insert
  """)
  print(f"Inserción de datos completada. {df_insert.count()} filas insertadas.")

except Exception as e:
    print(f"Error fatal: Fallo durante la carga de datos")
    print(f"Detalle del error: {e}")
    raise Exception("Error en TRUNCATE/INSERT. Verifique desfase de esquema.")


# Mover archivos procesados

In [0]:
try:
  import os
  from datetime import datetime

  # obtiene la fecha actual en formato yyyyMMdd (ej: 20251018)
  fecha_actual = datetime.now().strftime("%Y%m%d")

  for f in matched_files:
    # obtiene sólo el nombre del archivo (sin directorio)
    base_name = os.path.basename(f)

    # separa nombre y extensión
    name, ext = os.path.splitext(base_name)

    # construye el nuevo nombre agregando la fecha
    new_name = f"{name}_{fecha_actual}{ext}"

    # arma la ruta destino
    dest_path = os.path.join(path_processed, new_name)

    # mueve el archivo
    dbutils.fs.mv(f, dest_path)
    print(f"Archivo {base_name} movido a carpeta de archivos procesados")

except Exception as e:
    print(f"Error fatal: Fallo durante el movimiento de archivos procesados.")
    print(f"Detalle del error: {e}")
    raise Exception("Error al mover archivos (dbutils.fs.mv)")
